# Exercícios: Aprendizagem não supervisionada

In [ ]:
import os

import numpy as np
import pandas as pd

import sklearn
import sklearn.metrics
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme()

In [ ]:
# https://scikit-learn.org/stable/auto_examples/cluster/plot_agglomerative_dendrogram.html
from scipy.cluster.hierarchy import dendrogram
def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack(
        [model.children_, model.distances_, counts]
    ).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)

Para estes exercícios, será usado o conjunto de dados [Mall Customer Segmentation Data](https://www.kaggle.com/vjchoudhary7/customer-segmentation-tutorial-in-python):

In [ ]:
data_path = '../data/' if os.path.exists('../data/') else 'https://raw.githubusercontent.com/TheAwesomeGe/DECD/main/data/'
csv_file_path = data_path + 'Mall_Customers.csv'
df = pd.read_csv(csv_file_path, index_col='CustomerID')

1. Comece por analisar os dados e compreender os vários atributos.

In [ ]:
df.head()

In [ ]:
df.info()

2. Prepare os dados de forma a obter um conjunto que tenha apenas atributos numéricos. (**Nota**: Pode optar por descartar ou transformar os atributos que não sejam numéricos)

In [ ]:
prepared_df = df.drop(columns=['Gender'])
prepared_df.head()

3. Analise a distribuição de cada um dos atributos.

In [ ]:
prepared_df.describe()

In [ ]:
sns.pairplot(prepared_df);

4. Aplique a abordagem PCA e visualize a projeção dos dados nos dois componentes principais.

In [ ]:
pca = PCA(n_components=2)
pca_df = pd.DataFrame(pca.fit_transform(prepared_df), index=prepared_df.index, columns=['PC1', 'PC2'])
sns.scatterplot(pca_df, x='PC1', y='PC2');

5. Verifique qual a percentagem da variância do conjunto de dados explicada pelos dois componentes principais.

In [ ]:
pca.explained_variance_ratio_.sum()*100

6. Utilize o algoritmo *k-Means* para agrupar os dados. Escolha um valor de *k* adequado e visualize o agrupamento obtido.

In [ ]:
k_wss = pd.DataFrame([(k, KMeans(n_clusters=k, n_init='auto').fit(prepared_df).inertia_) for k in range(1, 11)], columns=['k', 'wss'])
sns.lineplot(k_wss, x='k', y='wss');

In [ ]:
kmeans = KMeans(n_clusters=5, n_init='auto').fit(prepared_df)
sns.scatterplot(pca_df, x='PC1', y='PC2', hue=kmeans.labels_);

7. Agrupe os dados usando *clustering* hierárquico aglomerativo e visualize o dendrograma.

In [ ]:
hierarchical_full = AgglomerativeClustering(n_clusters=None, distance_threshold=0).fit(prepared_df)
plot_dendrogram(hierarchical_full)

8. Selecione o mesmo número de clusters usado para o algoritmo *k-means* e compare os dois agrupamentos.

In [ ]:
hierarchical_k = AgglomerativeClustering(n_clusters=5).fit(prepared_df)
sns.scatterplot(pca_df, x='PC1', y='PC2', hue=kmeans.labels_).set(title='k-Means')
plt.show()
sns.scatterplot(pca_df, x='PC1', y='PC2', hue=hierarchical_k.labels_).set(title='Hierarchical Clustering')
plt.show()

9. Agrupe os dados usando o algoritmo *DBSCAN*. Explore e visualize os agrupamentos obtidos usando múltiplas parametrizações. Como varia o número de clusters e a percentagem de *outliers*?

In [ ]:
dbscan = DBSCAN(eps=15, min_samples=11).fit(prepared_df)
print(f'Number of clusters: {len(set(dbscan.labels_) - {-1})}')
print(f'Outliers: {sum(dbscan.labels_ == -1)*100 / len(dbscan.labels_)}%')
sns.scatterplot(pca_df, x='PC1', y='PC2', hue=dbscan.labels_);

10. Qual das três abordagens lhe parece produzir um agrupamento de melhor qualidade?

In [ ]:
eval_df = pd.DataFrame(index=['k-Means', 'Hierarchical Clustering', 'DBSCAN'])

eval_df['Silhouette Coefficient'] = [sklearn.metrics.silhouette_score(prepared_df, c.labels_) for c in (kmeans, hierarchical_k, dbscan)]
eval_df['Calinski-Harabasz Index'] = [sklearn.metrics.calinski_harabasz_score(prepared_df, c.labels_) for c in (kmeans, hierarchical_k, dbscan)]
eval_df['Davies-Bouldin Index'] = [sklearn.metrics.davies_bouldin_score(prepared_df, c.labels_) for c in (kmeans, hierarchical_k, dbscan)]

eval_df

11. Verifique se normalizar os dados tem impacto no resultado final.

In [ ]:
normalized_df = pd.DataFrame(StandardScaler().fit_transform(prepared_df), index=prepared_df.index, columns=prepared_df.columns)

kmeans_norm = KMeans(n_clusters=5, n_init='auto').fit(normalized_df)

print(f'Silhouette Coefficient: {sklearn.metrics.silhouette_score(normalized_df, kmeans_norm.labels_)}')
print(f'Calinski-Harabasz Index: {sklearn.metrics.calinski_harabasz_score(normalized_df, kmeans_norm.labels_)}')
print(f'Davies-Bouldin Index: {sklearn.metrics.davies_bouldin_score(normalized_df, kmeans_norm.labels_)}')

12. Discretize os atributos originais e repita as experiências. (**Sugestão**: Use as funções [`cut`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.cut.html) e [`qcut`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.qcut.html) da biblioteca *pandas*)

In [ ]:
discrete_df = pd.DataFrame(index=prepared_df.index)

discrete_df['Age'] = pd.cut(prepared_df['Age'], bins=[0, 18, 25, 45, 60, 70], labels=['minor', 'young', 'adult', 'experienced', 'elder'])
discrete_df['Income'] = pd.qcut(prepared_df['Annual Income (k$)'], q=3, labels=['low', 'medium', 'high'])
discrete_df['Spending Score'] = pd.cut(prepared_df['Spending Score (1-100)'], bins = 5, labels=['e', 'd', 'c', 'b', 'a'])

discrete_df.head()

In [ ]:
cat_num = {f: {c: i for i, c in enumerate(discrete_df[f].cat.categories)} for f in discrete_df.columns}

for f in discrete_df.columns:
    discrete_df[f] = discrete_df[f].replace(cat_num[f]).astype('int')
    
discrete_df.head()

In [ ]:
discrete_k_wss = pd.DataFrame([(k, KMeans(n_clusters=k, n_init='auto').fit(discrete_df).inertia_) for k in range(1, 11)], columns=['k', 'wss'])
sns.lineplot(discrete_k_wss, x='k', y='wss');

In [ ]:
discrete_kmeans = KMeans(n_clusters=4, n_init='auto').fit(discrete_df)
kmeans_df = discrete_df.copy()
kmeans_df['clusters'] = pd.Categorical(discrete_kmeans.labels_)
sns.pairplot(kmeans_df, hue='clusters');

In [ ]:
discrete_hierarchical = AgglomerativeClustering(n_clusters=4).fit(discrete_df)
hierarchical_df = discrete_df.copy()
hierarchical_df['clusters'] = pd.Categorical(discrete_hierarchical.labels_)
sns.pairplot(hierarchical_df, hue='clusters');

In [ ]:
discrete_dbscan = DBSCAN(eps=0.5, min_samples=11, metric='manhattan').fit(discrete_df)
dbscan_df = discrete_df.copy()
dbscan_df['clusters'] = pd.Categorical(discrete_dbscan.labels_)
sns.pairplot(dbscan_df, hue='clusters');

In [ ]:
discrete_eval_df = pd.DataFrame(index=['k-Means', 'Hierarchical Clustering', 'DBSCAN'])

discrete_eval_df['Silhouette Coefficient'] = [sklearn.metrics.silhouette_score(discrete_df, c) for c in (kmeans_df['clusters'], hierarchical_df['clusters'], dbscan_df['clusters'])]
discrete_eval_df['Calinski-Harabasz Index'] = [sklearn.metrics.calinski_harabasz_score(discrete_df, c) for c in (kmeans_df['clusters'], hierarchical_df['clusters'], dbscan_df['clusters'])]
discrete_eval_df['Davies-Bouldin Index'] = [sklearn.metrics.davies_bouldin_score(discrete_df, c) for c in (kmeans_df['clusters'], hierarchical_df['clusters'], dbscan_df['clusters'])]

discrete_eval_df